In [1]:
from math import exp
import numpy as np
from gym import spaces
import gym

In [2]:
# states is defined by the current level of the batterie and the time
# actions are defined by the charing rate (None, low, medium, high; 0-35)
alpha = [0.5, 0.3, 0.6, 0.2, 0.5, 0.8, 0.2, 0.1]
cap = 0 
cap_max = 70
p_mi = 0 
p_max = 35
p = 0
𝜇= 30
𝜎 = 5
penalty = -50

In [3]:
def cost_func(t, p):
    '''
    This method returns the costs for charging
    @param t:  time
    @param p:  charging rate
    '''
    return alpha[t] * exp(p)

In [4]:
def power_consumed(𝜇= 30, 𝜎 = 5, n=1):
    '''
    This method returns power consumed by the taxi driver
    @param 𝜇: mean
    @param 𝜎: standard deviation
    @param n: number of random variables
    '''
    return int(np.random.normal(𝜇,𝜎,n)[0])

In [5]:
print(power_consumed())

29


In [12]:
class SmartCharging(gym.Env):
    def __init__(self, actions, cap_max, penalty):
        super(SmartCharging, self).__init__()
        
        self.action_space = spaces.Discrete(actions)
        
        self.observation_space = spaces.Dict({
            'cap': spaces.Discrete(cap_max+1),
            'time': spaces.Discrete(8)
        })
        
        self.state = {
            'cap': 0,
            'time': 0
        }
        
    def reset(self,cap):
        self.state = {
            'cap' : cap,
            'time' : 0
        }
        
    def step(self,action):
        reward = 0
        if action + cap > cap_max:
            # if charing would exceed totally capacity cap is set to cap_max
            
            reward -= cost_func(self.state['time'], cap_max - cap)
            self.state['cap'] = cap_max
            self.state['time'] += 1
        else:
            reward -= cost_func(self.state['time'], action)
            
            self.state['cap'] += cap
            self.state['time'] += 1
        if self.state['time'] == 7:
            # if the final stage is reached reset the state
            
            consumed = power_consumed()
            if self.state['cap'] - consumed > 0:
                self.reset(self.state['cap'] - consumed)
            else:
                self.reset(0)
                reward -= 50

            
            
        return reward

In [20]:
env = SmartCharging(4,75, penalty)

In [29]:
env.action_space.sample()

1

In [ ]:
for episode in range(1000):
    env.reset(cap)
    
    